# Eurovision
*Problem Statement? Research Question?*


Datasets we have:
1. Contest Data
2. Contestants Data
3. Song Data 
4. Votes data
5. Betting Offices



In [1]:
# Import the different libraries
import pandas as pd

## 1. Contest Data


Key stats about the Contest Data
1. Shape of df (14,7)
2. Year column 2009 - 2023. No year missing. 
3. 14 countries. 13 unique countries & 1 country >1 count


### Importing Data

In [3]:
contest_df_raw = pd.read_csv('/Users/aylaabdullah/Desktop/bootcamp/Final project local work/Alex_Ayla_Neringa/data/contest_data.csv')

### Basic df stats

In [15]:
contest_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   year                             14 non-null     int64  
 1   host                             14 non-null     object 
 2   date                             14 non-null     object 
 3   semi_countries                   14 non-null     int64  
 4   final_countries                  14 non-null     int64  
 5   jury_countries_voting_final      9 non-null      float64
 6   televote_countries_voting_final  9 non-null      float64
dtypes: float64(2), int64(3), object(2)
memory usage: 916.0+ bytes


In [17]:
contest_df_raw.shape

(14, 7)

In [32]:
contest_df_raw.describe()

,year,semi_countries,final_countries,jury_countries_voting_final,televote_countries_voting_final
count,14.000000,14.000000,14.000000,9.000000,9.000000
mean,2015.714286,34.642857,25.785714,39.777778,39.888889
std,4.496641,2.205139,0.578934,2.438123,2.472066
min,2009.000000,31.000000,25.000000,36.000000,35.000000
25%,2012.250000,33.000000,25.250000,38.000000,39.000000
50%,2015.500000,35.000000,26.000000,40.000000,40.000000
75%,2018.750000,36.000000,26.000000,42.000000,42.000000
max,2023.000000,38.000000,27.000000,43.000000,43.000000


In [33]:
contest_df_raw.describe(include='object')

,host,date
count,14,14
unique,13,14
top,Sweden,13/05/2023
freq,2,1


### Columns

In [9]:
contest_df_raw.dtypes

year                                 int64
host                                object
date                                object
semi_countries                       int64
final_countries                      int64
jury_countries_voting_final        float64
televote_countries_voting_final    float64
dtype: object

In [21]:
contest_df_raw

,year,host,date,semi_countries,final_countries,jury_countries_voting_final,televote_countries_voting_final
0,2023,United Kingdom,13/05/2023,31,26,37.0,38.0
1,2022,Italy,14/05/2022,35,25,40.0,40.0
2,2021,Netherlands,22/05/2021,33,26,39.0,39.0
3,2019,Israel,18/05/2019,35,26,41.0,41.0
4,2018,Portugal,12/05/2018,37,26,43.0,43.0
5,2017,Ukraine,13/05/2017,36,26,42.0,42.0
6,2016,Sweden,14/05/2016,36,26,42.0,42.0
7,2015,Austria,23/05/2015,33,27,38.0,39.0
8,2014,Denmark,10/05/2014,31,26,36.0,35.0
9,2013,Sweden,18/05/2013,33,26,NaN,NaN


#### Year info

In [27]:
print(f'Start year: {contest_df_raw['year'].min()}')
print(f'End year: {contest_df_raw['year'].max()}')
print(f'Years missing: {(2023-2009) - contest_df_raw['year'].nunique()}')
print(f'Column type: {contest_df_raw['year'].dtype}')


Start year: 2009
End year: 2023
Years missing: 0
Column type: int64


#### Host info

In [28]:
contest_df_raw['host']

0     United Kingdom
1              Italy
2        Netherlands
3             Israel
4           Portugal
5            Ukraine
6             Sweden
7            Austria
8            Denmark
9             Sweden
10        Azerbaijan
11           Germany
12            Norway
13            Russia
Name: host, dtype: object

In [45]:
print(f'List of countries and unique count: {contest_df_raw['host'].value_counts()}')
print(f'Number of countries appearing >1: {((contest_df_raw['host'].count())>1).sum()}')
print(f'Number of unique countries: {contest_df_raw['host'].nunique()}')


List of countries and unique count: host
Sweden            2
United Kingdom    1
Italy             1
Netherlands       1
Israel            1
Portugal          1
Ukraine           1
Austria           1
Denmark           1
Azerbaijan        1
Germany           1
Norway            1
Russia            1
Name: count, dtype: int64
Number of countries appearing >1: 1
Number of unique countries: 13


#### Date info:
_have to fix date column type_

#### semi_countries

#### final_countries

#### jury countries voting final

#### televote countries voting final

### Rows

In [13]:
contest_df_raw.index

RangeIndex(start=0, stop=14, step=1)

## 2. Contestants Data

## 3. Song Data

## 4. Votes Data

## 5. Betting Offices 